In [1]:
'''
Reader for EDF+ files.
TODO:
 - add support for log-transformed channels:
   http://www.edfplus.info/specs/edffloat.html and test with
   data generated with
   http://www.edfplus.info/downloads/software/NeuroLoopGain.zip.
 - check annotations with Schalk's Physiobank data.
Copyright (c) 2012 Boris Reuderink.
'''

import re, datetime, operator, logging
import numpy as np
from collections import namedtuple

EVENT_CHANNEL = 'EDF Annotations'
log = logging.getLogger(__name__)

class EDFEndOfData: pass


def tal(tal_str):
  '''Return a list with (onset, duration, annotation) tuples for an EDF+ TAL
  stream.
  '''
  exp = '(?P<onset>[+\-]\d+(?:\.\d*)?)' + \
    '(?:\x15(?P<duration>\d+(?:\.\d*)?))?' + \
    '(\x14(?P<annotation>[^\x00]*))?' + \
    '(?:\x14\x00)'

  def annotation_to_list(annotation):
    return unicode(annotation, 'utf-8').split('\x14') if annotation else []

  def parse(dic):
    return (
      float(dic['onset']),
      float(dic['duration']) if dic['duration'] else 0.,
      annotation_to_list(dic['annotation']))

  return [parse(m.groupdict()) for m in re.finditer(exp, tal_str)]


def edf_header(f):
  h = {}
  assert f.tell() == 0  # check file position
  assert f.read(8) == '0       '

  # recording info)
  h['local_subject_id'] = f.read(80).strip()
  h['local_recording_id'] = f.read(80).strip()

  # parse timestamp
  (day, month, year) = [int(x) for x in re.findall('(\d+)', f.read(8))]
  (hour, minute, sec)= [int(x) for x in re.findall('(\d+)', f.read(8))]
  h['date_time'] = str(datetime.datetime(year + 2000, month, day,
    hour, minute, sec))

  # misc
  header_nbytes = int(f.read(8))
  subtype = f.read(44)[:5]
  h['EDF+'] = subtype in ['EDF+C', 'EDF+D']
  h['contiguous'] = subtype != 'EDF+D'
  h['n_records'] = int(f.read(8))
  h['record_length'] = float(f.read(8))  # in seconds
  nchannels = h['n_channels'] = int(f.read(4))

  # read channel info
  channels = range(h['n_channels'])
  h['label'] = [f.read(16).strip() for n in channels]
  h['transducer_type'] = [f.read(80).strip() for n in channels]
  h['units'] = [f.read(8).strip() for n in channels]
  h['physical_min'] = np.asarray([float(f.read(8)) for n in channels])
  h['physical_max'] = np.asarray([float(f.read(8)) for n in channels])
  h['digital_min'] = np.asarray([float(f.read(8)) for n in channels])
  h['digital_max'] = np.asarray([float(f.read(8)) for n in channels])
  h['prefiltering'] = [f.read(80).strip() for n in channels]
  h['n_samples_per_record'] = [int(f.read(8)) for n in channels]
  f.read(32 * nchannels)  # reserved

  assert f.tell() == header_nbytes
  return h


class BaseEDFReader:
  def __init__(self, file):
    self.file = file


  def read_header(self):
    self.header = h = edf_header(self.file)

    # calculate ranges for rescaling
    self.dig_min = h['digital_min']
    self.phys_min = h['physical_min']
    phys_range = h['physical_max'] - h['physical_min']
    dig_range = h['digital_max'] - h['digital_min']
    assert np.all(phys_range > 0)
    assert np.all(dig_range > 0)
    self.gain = phys_range / dig_range


  def read_raw_record(self):
    '''Read a record with data and return a list containing arrays with raw
    bytes.
    '''
    result = []
    for nsamp in self.header['n_samples_per_record']:
      samples = self.file.read(nsamp * 2)
      if len(samples) != nsamp * 2:
        raise EDFEndOfData
      result.append(samples)
    return result


  def convert_record(self, raw_record):
    '''Convert a raw record to a (time, signals, events) tuple based on
    information in the header.
    '''
    h = self.header
    dig_min, phys_min, gain = self.dig_min, self.phys_min, self.gain
    time = float('nan')
    signals = []
    events = []
    for (i, samples) in enumerate(raw_record):
      if h['label'][i] == EVENT_CHANNEL:
        ann = tal(samples)
        time = ann[0][0]
        events.extend(ann[1:])
        # print(i, samples)
        # exit()
      else:
        # 2-byte little-endian integers
        dig = np.fromstring(samples, '<i2').astype(np.float32)
        phys = (dig - dig_min[i]) * gain[i] + phys_min[i]
        signals.append(phys)

    return time, signals, events


  def read_record(self):
    return self.convert_record(self.read_raw_record())


  def records(self):
    '''
    Record generator.
    '''
    try:
      while True:
        yield self.read_record()
    except EDFEndOfData:
      pass


def load_edf(edffile):
  '''Load an EDF+ file.
  Very basic reader for EDF and EDF+ files. While BaseEDFReader does support
  exotic features like non-homogeneous sample rates and loading only parts of
  the stream, load_edf expects a single fixed sample rate for all channels and
  tries to load the whole file.
  Parameters
  ----------
  edffile : file-like object or string
  Returns
  -------
  Named tuple with the fields:
    X : NumPy array with shape p by n.
      Raw recording of n samples in p dimensions.
    sample_rate : float
      The sample rate of the recording. Note that mixed sample-rates are not
      supported.
    sens_lab : list of length p with strings
      The labels of the sensors used to record X.
    time : NumPy array with length n
      The time offset in the recording for each sample.
    annotations : a list with tuples
      EDF+ annotations are stored in (start, duration, description) tuples.
      start : float
        Indicates the start of the event in seconds.
      duration : float
        Indicates the duration of the event in seconds.
      description : list with strings
        Contains (multiple?) descriptions of the annotation event.
  '''
  if isinstance(edffile, basestring):
    with open(edffile, 'rb') as f:
      return load_edf(f)  # convert filename to file

  reader = BaseEDFReader(edffile)
  reader.read_header()

  h = reader.header
  log.debug('EDF header: %s' % h)

  # get sample rate info
  nsamp = np.unique(
    [n for (l, n) in zip(h['label'], h['n_samples_per_record'])
    if l != EVENT_CHANNEL])
  assert nsamp.size == 1, 'Multiple sample rates not supported!'
  sample_rate = float(nsamp[0]) / h['record_length']

  rectime, X, annotations = zip(*reader.records())
  X = np.hstack(X)
  annotations = reduce(operator.add, annotations)
  chan_lab = [lab for lab in reader.header['label'] if lab != EVENT_CHANNEL]

  # create timestamps
  if reader.header['contiguous']:
    time = np.arange(X.shape[1]) / sample_rate
  else:
    reclen = reader.header['record_length']
    within_rec_time = np.linspace(0, reclen, nsamp, endpoint=False)
    time = np.hstack([t + within_rec_time for t in rectime])

  tup = namedtuple('EDF', 'X sample_rate chan_lab time annotations')
  return tup(X, sample_rate, chan_lab, time, annotations)

In [3]:
import numpy as np
channel_map=np.zeros(shape=(10,11),dtype=np.int32)
channel_map[0,4:7]=range(22,25)
channel_map[1,3:8]=range(25,30)
channel_map[2,1:10]=range(30,39)
channel_map[3,1]=39
channel_map[3,9]=40
channel_map[3,2:9]=range(1,8)
channel_map[4,0]=43
channel_map[4,1]=41
channel_map[4,9]=42
channel_map[4,10]=44
channel_map[4,2:9]=range(8,15)
channel_map[5,1]=45
channel_map[5,9]=46
channel_map[5,2:9]=range(15,22)
channel_map[6,1:10]=range(47,56)
channel_map[9,5]=64
channel_map[8,4:7]=range(61,64)
channel_map[7,3:8]=range(56,61)
channel_map

array([[ 0,  0,  0,  0, 22, 23, 24,  0,  0,  0,  0],
       [ 0,  0,  0, 25, 26, 27, 28, 29,  0,  0,  0],
       [ 0, 30, 31, 32, 33, 34, 35, 36, 37, 38,  0],
       [ 0, 39,  1,  2,  3,  4,  5,  6,  7, 40,  0],
       [43, 41,  8,  9, 10, 11, 12, 13, 14, 42, 44],
       [ 0, 45, 15, 16, 17, 18, 19, 20, 21, 46,  0],
       [ 0, 47, 48, 49, 50, 51, 52, 53, 54, 55,  0],
       [ 0,  0,  0, 56, 57, 58, 59, 60,  0,  0,  0],
       [ 0,  0,  0,  0, 61, 62, 63,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 64,  0,  0,  0,  0,  0]], dtype=int32)

In [4]:
win_size=10
sample_win_matrix_0=[] #[num,time_steps,10,11]
sample_win_matrix_1=[]
def label(name):
    if(name[0]=='T0'):
        return 0
    if((name[0]=='T1')|(name[0]=='T2')):
        return 1
    print name
    print "problem!"
    return 0
    
for person in range(1,90): 
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)

    data=load_edf("../S"+index_person+"R01.edf")
    X=data[0]
    num_wins=int((X.shape[1]-win_size)/5)+1
    
    num_win=0
    while(num_win<num_wins):
        win=X[:,num_win*5:num_win*5+win_size]
        win_matrix=[] #np.zeros(shape=(win_size,10,11),dtype=np.float32)
        for i in range(win_size):
            matrix_i=np.zeros(shape=(10,11),dtype=np.float32)
            for x in range(10):
                for y in range(11):
                    if (channel_map[x,y]!=0):
                        matrix_i[x,y]=win[channel_map[x,y]-1,i]
                    else :
                        matrix_i[x,y]=0.0
            win_matrix.append(matrix_i)
        sample_win_matrix_0.append(win_matrix)
               
        num_win+=1
   

    ### task 1:open and close left or right fist
    data=load_edf("../S"+index_person+"R03.edf")
    X=data[0]
    time=data[3]
    annotation=data[4]
    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            win=X[:,i:i+win_size]
            win_matrix=[] 
            for j in range(win_size):
                matrix_j=np.zeros(shape=(10,11),dtype=np.float32)
                for x in range(10):
                    for y in range(11):
                        if(channel_map[x,y]!=0):
                             matrix_j[x,y]=win[channel_map[x,y]-1,j]
                        else :
                             matrix_j[x,y]=0.0
                win_matrix.append(matrix_j)
            sample_win_matrix_0.append(win_matrix)
            i+=5
            if(i+win_size-1>=i_max):
                break
            


    #task 2 motion imagery 
    data=load_edf("../S"+index_person+"R04.edf")
    X=data[0]
    time=data[3]
    annotation=data[4]
    data=load_edf("../S"+index_person+"R08.edf")
    X=np.concatenate((X,data[0]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("../S"+index_person+"R12.edf")
    X=np.concatenate((X,data[0]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]

    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            win=X[:,i:i+win_size]
            win_matrix=[] 
            for j in range(win_size):
                matrix_j=np.zeros(shape=(10,11),dtype=np.float32)
                for x in range(10):
                    for y in range(11):
                        if(channel_map[x,y]!=0):
                            matrix_j[x,y]=win[channel_map[x,y]-1,j]
                        else:
                            matrix_j[x,y]=0.0
                win_matrix.append(matrix_j)
            if(label(annot[2])==0):
                sample_win_matrix_0.append(win_matrix)
            else :
                sample_win_matrix_1.append(win_matrix)
            
            i+=5
            if(i+win_size-1>=i_max):
                break

                
sample_win_matrix_0=np.array(sample_win_matrix_0)
sample_win_matrix_1=np.array(sample_win_matrix_1)

IOError: [Errno 2] No such file or directory: '../S002R01.edf'

In [12]:
import tensorflow as tf
#variables
tf.reset_default_graph()
batch_size=222
num_channels=8
num_frequency=22

channel_frequency_input=tf.placeholder(tf.float32,shape=(222,num_channels,num_frequency,1)) #[batch, in_height=channels, in_width=frequency, in_channels=1]
encoder_labels=tf.placeholder(tf.int32)
# filter1=tf.Variable(tf.random_uniform((2,3,1,32),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter2=tf.Variable(tf.random_uniform((2,3,32,16),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter3=tf.Variable(tf.random_uniform((4,6,16,8),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
filter1=tf.Variable(tf.random_uniform((1,2,1,32),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
filter2=tf.Variable(tf.random_uniform((1,2,32,16),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
filter3=tf.Variable(tf.random_uniform((8,8,16,8),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter1=tf.Variable(tf.random_uniform((3,3,1,16),minval=0.0000001,maxval=0.00001,dtype=tf.float32))
# filter2=tf.Variable(tf.random_uniform((4,8,16,8),minval=0.0000001,maxval=0.00001,dtype=tf.float32))

conv1=tf.nn.conv2d(
    input=channel_frequency_input,
    filter=filter1, #[filter_height, filter_width, in_channels, out_channels]
    strides=[1,1,2,1],           #[1, stride, stride, 1]
    padding="VALID",
    #use_cudnn_on_gpu=True,
    data_format='NHWC'
)

pool1=tf.nn.max_pool(
    value=conv1,
    ksize=[1,1,2,1],
    strides=[1,1,1,1],
    padding='VALID',
    data_format='NHWC'
)

conv2=tf.nn.conv2d(
    input=pool1,
    filter=filter2, #[filter_height, filter_width, in_channels, out_channels]
    strides=[1,1,1,1],           #[1, stride, stride, 1]
    padding="VALID",
    #use_cudnn_on_gpu=True,
    data_format='NHWC'
)

pool2=tf.nn.max_pool(
    value=conv2,
    ksize=[1,1,2,1],
    strides=[1,1,1,1],
    padding='VALID',
    data_format='NHWC'
)

conv3=tf.nn.conv2d(
    input=pool2,
    filter=filter3, #[filter_height, filter_width, in_channels, out_channels]
    strides=[1,1,1,1],           #[1, stride, stride, 1]
    padding="VALID",
    #use_cudnn_on_gpu=True,
    data_format='NHWC'
)

flat_cnn=tf.contrib.layers.flatten(conv3)
#flat_cnn=tf.contrib.layers.flatten(conv2)


output_cnn=tf.sigmoid(flat_cnn)
output_shape1=tf.size(output_cnn)

#LSTM
num_time_steps=20
channel_amplitude_input=[]
for i in range(0,num_time_steps):
    channel_amplitude_input.append(tf.placeholder(tf.float32,shape=(222,num_channels)))


cell = tf.nn.rnn_cell.BasicLSTMCell(
    num_units=16, 
    forget_bias=1.0, 
    state_is_tuple=True)

cell2 = tf.nn.rnn_cell.BasicLSTMCell(
    num_units=8, 
    forget_bias=1.0, 
    state_is_tuple=True)

lstm_cell = tf.contrib.rnn.MultiRNNCell(cells=[cell,cell2], state_is_tuple=True)


output,final_state= tf.nn.static_rnn(
    cell=lstm_cell,
    #cell=cell2,
    inputs=channel_amplitude_input,
    #A length T list of inputs,
    #each a Tensor of shape [batch_size, input_size], 
    #or a nested tuple of such elements.
    initial_state=None,
    dtype=tf.float32,
    sequence_length=tf.convert_to_tensor(np.repeat(num_time_steps,batch_size)),
    scope=None
)

output_shape2=tf.size(output[-1])
flat_lstm=tf.contrib.layers.flatten(output[-1])

vec_concat=tf.concat([output_cnn,flat_lstm],axis=1)


W=tf.Variable(tf.random_uniform((16,2),minval=0.0000001,maxval=0.00001,dtype=tf.float32)) 
logits=tf.matmul(vec_concat,W)
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=encoder_labels, logits=logits)
train_loss = (tf.reduce_sum(crossent)/batch_size)

# Optimization
params = tf.trainable_variables()# the list of params to be trained
print params #check the trainable params, including encoder's ?

optimizer = tf.train.AdamOptimizer(learning_rate=0.001,epsilon=0.001)
gradients=optimizer.compute_gradients(
    train_loss,
    var_list=None,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)

update= optimizer.apply_gradients(gradients)


tf.summary.scalar('LLLLoss', train_loss)


for i,g in enumerate(gradients):
    tf.summary.histogram('GGGGradients'+str(i), g[0])
    
merged = tf.summary.merge_all()
sess=tf.Session()
train_writer = tf.summary.FileWriter('/home/weiyizhang/brain/experiment/summary',sess.graph)

init = tf.global_variables_initializer()
sess.run(init)
# channel_amp=[]
# for i in range(10):
#     channel_amp.append(np.zeros(shape=(222,8),dtype="float32"))

# g,s,s2,l,_,summ=sess.run([gradients,output_shape1,output_shape2,train_loss,update,merged],{channel_frequency_input:np.zeros(shape=(222,8,12,1),dtype="float32"),\
#                           encoder_labels:np.repeat(1,222),
#                           channel_amplitude_input[0]:channel_amp[0],  
#                           channel_amplitude_input[1]:channel_amp[1], 
#                           channel_amplitude_input[2]:channel_amp[2],
#                           channel_amplitude_input[3]:channel_amp[3], 
#                           channel_amplitude_input[4]:channel_amp[4], 
#                           channel_amplitude_input[5]:channel_amp[5], 
#                           channel_amplitude_input[6]:channel_amp[6], 
#                           channel_amplitude_input[7]:channel_amp[7], 
#                           channel_amplitude_input[8]:channel_amp[8], 
#                           channel_amplitude_input[9]:channel_amp[9]})
# writer.add_summary(summ, 1)
# writer.add_summary(summ, 2)
# print type(g)
# print g[0]
# print s
# print s2
# print l

[<tf.Variable 'Variable:0' shape=(1, 2, 1, 32) dtype=float32_ref>, <tf.Variable 'Variable_1:0' shape=(1, 2, 32, 16) dtype=float32_ref>, <tf.Variable 'Variable_2:0' shape=(8, 8, 16, 8) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(24, 64) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(24, 32) dtype=float32_ref>, <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'Variable_3:0' shape=(16, 2) dtype=float32_ref>]


In [13]:
channel_available=[60,62,31,35,48,52,8,12] #index of Channels availables
win_size=20
#data for test
win_channel_sample_test=[]
win_channel_fft_test=[]
win_label_test=[]
i_summary=0
for person in range(1,93):
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)
#     print "num person"  
#     print index_person     
    ###baseline data of eye activity
    #no step, all the same
    data=load_edf("S"+index_person+"R01.edf")
    X=data[0][channel_available]
#     data=load_edf("S"+index_person+"R02.edf")
#     X=np.concatenate((X,data[0][channel_available]),axis=1)
#     print "baseline num"
#     print X.shape
    num_wins=int((X.shape[1]-win_size)/5)+1
    win_channel_sample_baseline=[]
    win_channel_fft_baseline=[]
    #(batch_size,num channels,len of fft=len of sample =10)
    num_win=0
    while(num_win<num_wins):
        win_channel_sample_baseline.append(X[:,num_win*5:num_win*5+win_size])
        num_win+=1
    for num,channel_sample in enumerate(win_channel_sample_baseline):
        real=np.fft.fft(channel_sample,axis=1).real[:,:11]
        image=np.fft.fft(channel_sample,axis=1).imag[:,:11]
        c=np.zeros(shape=(len(channel_available),22),dtype=np.float32)
        c[:,::2]=image
        c[:,1::2]=real
        win_channel_fft_baseline.append(c)
    win_label_baseline=np.repeat(0,num_wins).tolist()#0 represent baseline
    win_channel_sample_baseline=np.array(win_channel_sample_baseline,dtype="float32")
    win_channel_fft_baseline=np.array(win_channel_fft_baseline,dtype="float32")
#     print "baseline num final"
#     print win_channel_sample_baseline.shape

    ### task 1:open and close left or right fist
    data=load_edf("S"+index_person+"R03.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
#     data=load_edf("S"+index_person+"R07.edf")
#     X=np.concatenate((X,data[0][channel_available]),axis=1)
#     time=np.concatenate((time,data[3]),axis=0)
#     annotation+=data[4]
#     data=load_edf("S"+index_person+"R11.edf")
#     X=np.concatenate((X,data[0][channel_available]),axis=1)
#     time=np.concatenate((time,data[3]),axis=0)
#     annotation+=data[4]
#     print "task1 num"
#     print X.shape
#     print time.shape
    win_channel_sample_task1=[]
    win_channel_fft_task1=[]
    win_label_task1=[]
    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            win_channel_sample_task1.append(X[:,i:i+win_size])
            win_label_task1.append(0)
            i+=5
            if(i+win_size-1>=i_max):
                break
            

    win_channel_sample_task1=np.array(win_channel_sample_task1,dtype="float32")
    for channel_sample in win_channel_sample_task1:
        real=np.fft.fft(channel_sample,axis=1).real[:,:11]
        image=np.fft.fft(channel_sample,axis=1).imag[:,:11]
        c=np.zeros(shape=(len(channel_available),22),dtype=np.float32)
        c[:,::2]=image
        c[:,1::2]=real
        win_channel_fft_task1.append(c)  
    win_channel_fft_task1=np.array(win_channel_fft_task1,dtype="float32")
#     print "task1 num final"
#     print win_channel_sample_task1.shape
    ###task 2 : imagery
    def label(name):
        if(name[0]=='T0'):
            return 0
        if((name[0]=='T1')|(name[0]=='T2')):
            return 1
        print name
        print "problem!"
        return 0

    data=load_edf("S"+index_person+"R04.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
    data=load_edf("S"+index_person+"R08.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("S"+index_person+"R12.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
#     print "task2.shape"
#     print X.shape
#     print time.shape
    win_channel_sample_task2=[]
    win_channel_fft_task2=[]
    win_label_task2=[]
    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+win_size-1]<=annot[0]+annot[1])&(time[i+win_size-1]>annot[0])):
            win_channel_sample_task2.append(X[:,i:i+win_size])
            win_label_task2.append(label(annot[2]))
            i+=5
            if(i+win_size-1>=i_max):
                break
            
    win_channel_sample_task2=np.array(win_channel_sample_task2,dtype="float32")
    for channel_sample in win_channel_sample_task2:
        real=np.fft.fft(channel_sample,axis=1).real[:,:11]
        image=np.fft.fft(channel_sample,axis=1).imag[:,:11]
        c=np.zeros(shape=(len(channel_available),22),dtype=np.float32)
        c[:,::2]=image
        c[:,1::2]=real
        win_channel_fft_task2.append(c) 
    win_channel_fft_task2=np.array(win_channel_fft_task2,dtype="float32")
#     print "task2 num final"
#     print win_channel_sample_task2.shape
    #input to network
    win_channel_sample=np.concatenate((win_channel_sample_baseline,win_channel_sample_task1,win_channel_sample_task2),axis=0)
    win_channel_fft=np.concatenate((win_channel_fft_baseline,win_channel_fft_task1,win_channel_fft_task2),axis=0)
    win_channel_fft=np.expand_dims(win_channel_fft,axis=3)
    win_label=np.array(win_label_baseline+win_label_task1+win_label_task2)
    
    num_window=win_channel_sample.shape[0]
    random_index=range(num_window)
    np.random.shuffle(random_index)
    win_channel_sample=win_channel_sample[random_index]
    win_channel_fft=win_channel_fft[random_index]
    win_label=win_label[random_index]
    
#     print "win_channel_sample"
#     print win_channel_sample.shape
#     print "win_cahnnel_fft"
#     print win_channel_fft.shape
#     print "win_label"
#     print win_label.shape
#     print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
    
    num_batch=int(win_label.shape[0]/222)-1
    for i in range(num_batch):
        win_channel_sample_batch=win_channel_sample[i*222:(i+1)*222,:,:]
        sample_channel_win_batch=np.swapaxes(win_channel_sample_batch,0,2)
        sample_win_channel_batch=np.swapaxes(sample_channel_win_batch,1,2)
        sample_win_channel_batch=sample_win_channel_batch.tolist()
        win_channel_fft_batch=win_channel_fft[i*222:(i+1)*222,:,:,:]
        win_label_batch=win_label[i*222:(i+1)*222]
        c,_,summary=sess.run([crossent,update,merged],{channel_frequency_input:win_channel_fft_batch,
                          encoder_labels:win_label_batch,
                          channel_amplitude_input[0]:sample_win_channel_batch[0],  
                          channel_amplitude_input[1]:sample_win_channel_batch[1], 
                          channel_amplitude_input[2]:sample_win_channel_batch[2],
                          channel_amplitude_input[3]:sample_win_channel_batch[3], 
                          channel_amplitude_input[4]:sample_win_channel_batch[4], 
                          channel_amplitude_input[5]:sample_win_channel_batch[5], 
                          channel_amplitude_input[6]:sample_win_channel_batch[6], 
                          channel_amplitude_input[7]:sample_win_channel_batch[7], 
                          channel_amplitude_input[8]:sample_win_channel_batch[8], 
                          channel_amplitude_input[9]:sample_win_channel_batch[9],
                          channel_amplitude_input[10]:sample_win_channel_batch[10],  
                          channel_amplitude_input[11]:sample_win_channel_batch[11], 
                          channel_amplitude_input[12]:sample_win_channel_batch[12],
                          channel_amplitude_input[13]:sample_win_channel_batch[13], 
                          channel_amplitude_input[14]:sample_win_channel_batch[14], 
                          channel_amplitude_input[15]:sample_win_channel_batch[15], 
                          channel_amplitude_input[16]:sample_win_channel_batch[16], 
                          channel_amplitude_input[17]:sample_win_channel_batch[17], 
                          channel_amplitude_input[18]:sample_win_channel_batch[18], 
                          channel_amplitude_input[19]:sample_win_channel_batch[19]})
        train_writer.add_summary(summary, i_summary)
        i_summary+=1
        
    #rest data used for test  
    win_channel_sample_test+=(win_channel_sample[num_batch*222:,:,:]).tolist() 
    win_channel_fft_test+=(win_channel_fft[num_batch*222:,:,:,:]).tolist()
    win_label_test+=(win_label[num_batch*222:]).tolist()

    
win_channel_sample_test=np.array(win_channel_sample_test,dtype='float32')
sample_channel_win_test=np.swapaxes(win_channel_sample_test,0,2)
sample_win_channel_test=np.swapaxes(sample_channel_win_test,1,2)
sample_win_channel_test=sample_win_channel_test.tolist()

win_channel_fft_test=np.array(win_channel_fft_test,dtype='float32')
win_label_test=np.array(win_label_test)


prob=tf.nn.softmax(logits)
predict=tf.argmax(prob,axis=1,output_type=tf.int32)
result=[]
pro=[]
for i in range (int(win_label_test.shape[0]/222)):
    p,r=sess.run([prob,predict],{channel_frequency_input:win_channel_fft_test[222*i:222*i+222,:,:,:],
                          channel_amplitude_input[0]:sample_win_channel_test[0][222*i:222*i+222],  
                          channel_amplitude_input[1]:sample_win_channel_test[1][222*i:222*i+222], 
                          channel_amplitude_input[2]:sample_win_channel_test[2][222*i:222*i+222],
                          channel_amplitude_input[3]:sample_win_channel_test[3][222*i:222*i+222], 
                          channel_amplitude_input[4]:sample_win_channel_test[4][222*i:222*i+222], 
                          channel_amplitude_input[5]:sample_win_channel_test[5][222*i:222*i+222], 
                          channel_amplitude_input[6]:sample_win_channel_test[6][222*i:222*i+222], 
                          channel_amplitude_input[7]:sample_win_channel_test[7][222*i:222*i+222], 
                          channel_amplitude_input[8]:sample_win_channel_test[8][222*i:222*i+222], 
                          channel_amplitude_input[9]:sample_win_channel_test[9][222*i:222*i+222],
                          channel_amplitude_input[10]:sample_win_channel_test[10][222*i:222*i+222],  
                          channel_amplitude_input[11]:sample_win_channel_test[11][222*i:222*i+222], 
                          channel_amplitude_input[12]:sample_win_channel_test[12][222*i:222*i+222],
                          channel_amplitude_input[13]:sample_win_channel_test[13][222*i:222*i+222], 
                          channel_amplitude_input[14]:sample_win_channel_test[14][222*i:222*i+222], 
                          channel_amplitude_input[15]:sample_win_channel_test[15][222*i:222*i+222], 
                          channel_amplitude_input[16]:sample_win_channel_test[16][222*i:222*i+222], 
                          channel_amplitude_input[17]:sample_win_channel_test[17][222*i:222*i+222], 
                          channel_amplitude_input[18]:sample_win_channel_test[18][222*i:222*i+222], 
                          channel_amplitude_input[19]:sample_win_channel_test[19][222*i:222*i+222]})
    result+=r.tolist()
    pro+=p.tolist()
print type(result)
print type(win_label_test)
from sklearn.metrics import classification_report
print classification_report(win_label_test[:len(result)],np.array(result))
print [pro[i][1] for i in range(len(pro)) if win_label_test[i]==1]

<type 'list'>
<type 'numpy.ndarray'>
             precision    recall  f1-score   support

          0       0.70      0.59      0.64     25570
          1       0.38      0.50      0.43     12836

avg / total       0.59      0.56      0.57     38406

[0.4298037588596344, 0.5140441060066223, 0.5119946599006653, 0.5365142822265625, 0.4224375784397125, 0.5812546014785767, 0.4400773048400879, 0.47967737913131714, 0.4011346995830536, 0.41274121403694153, 0.5163099765777588, 0.41825705766677856, 0.4787546396255493, 0.43908950686454773, 0.494824081659317, 0.46568962931632996, 0.4816421866416931, 0.49713853001594543, 0.47621607780456543, 0.547555685043335, 0.5147570371627808, 0.4424138367176056, 0.4568891227245331, 0.4623173475265503, 0.4707877039909363, 0.5166646242141724, 0.4480960965156555, 0.5201794505119324, 0.5089864730834961, 0.49618247151374817, 0.4893699884414673, 0.4954155385494232, 0.5392640233039856, 0.5203821659088135, 0.5336184501647949, 0.4492836892604828, 0.5177010893821716, 0

In [138]:
# win_channel_sample_test=np.array(win_channel_sample_test,dtype='float32')
# sample_channel_win_test=np.swapaxes(win_channel_sample_test,0,2)
# sample_win_channel_test=np.swapaxes(sample_channel_win_test,1,2)
# sample_win_channel_test=sample_win_channel_test.tolist()
# win_channel_fft_test=np.array(win_channel_fft_test,dtype='float32')
# win_label_test=np.array(win_label_test)

# prob=tf.nn.softmax(logits)
# predict=tf.argmax(prob,axis=1,output_type=tf.int32)
# result=[]
# pro=[]
# for i in range (int(win_label_test.shape[0]/222)):
#     p,r=sess.run([prob,predict],{channel_frequency_input:win_channel_fft_test[222*i:222*i+222,:,:,:],
#                           channel_amplitude_input[0]:sample_win_channel_test[0][222*i:222*i+222],  
#                           channel_amplitude_input[1]:sample_win_channel_test[1][222*i:222*i+222], 
#                           channel_amplitude_input[2]:sample_win_channel_test[2][222*i:222*i+222],
#                           channel_amplitude_input[3]:sample_win_channel_test[3][222*i:222*i+222], 
#                           channel_amplitude_input[4]:sample_win_channel_test[4][222*i:222*i+222], 
#                           channel_amplitude_input[5]:sample_win_channel_test[5][222*i:222*i+222], 
#                           channel_amplitude_input[6]:sample_win_channel_test[6][222*i:222*i+222], 
#                           channel_amplitude_input[7]:sample_win_channel_test[7][222*i:222*i+222], 
#                           channel_amplitude_input[8]:sample_win_channel_test[8][222*i:222*i+222], 
#                           channel_amplitude_input[9]:sample_win_channel_test[9][222*i:222*i+222],
#                           channel_amplitude_input[10]:sample_win_channel_test[10][222*i:222*i+222],  
#                           channel_amplitude_input[11]:sample_win_channel_test[11][222*i:222*i+222], 
#                           channel_amplitude_input[12]:sample_win_channel_test[12][222*i:222*i+222],
#                           channel_amplitude_input[13]:sample_win_channel_test[13][222*i:222*i+222], 
#                           channel_amplitude_input[14]:sample_win_channel_test[14][222*i:222*i+222], 
#                           channel_amplitude_input[15]:sample_win_channel_test[15][222*i:222*i+222], 
#                           channel_amplitude_input[16]:sample_win_channel_test[16][222*i:222*i+222], 
#                           channel_amplitude_input[17]:sample_win_channel_test[17][222*i:222*i+222], 
#                           channel_amplitude_input[18]:sample_win_channel_test[18][222*i:222*i+222], 
#                           channel_amplitude_input[19]:sample_win_channel_test[19][222*i:222*i+222]})
#     result+=r.tolist()
#     pro+=p.tolist()
thresh=[]
for p in pro:
    if (p[1]>0.54):
        thresh.append(1)
    else:
        thresh.append(0)
from sklearn.metrics import classification_report
print classification_report(win_label_test[:len(thresh)],np.array(thresh))
#print [pro[i][1] for i in range(len(pro)) if win_label_test[i]==1]

             precision    recall  f1-score   support

          0       0.68      0.82      0.74     25481
          1       0.39      0.23      0.29     12925

avg / total       0.58      0.62      0.59     38406



In [108]:
print classification_report(win_label_test[:len(result)],np.array(result))

             precision    recall  f1-score   support

          0       0.68      0.80      0.73     16658
          1       0.38      0.25      0.30      8428

avg / total       0.58      0.61      0.59     25086



In [5]:
for person in range(1,110):
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)
    print "num person"  
    print index_person     
    ###baseline data of eye activity
    #no step, all the same
    data=load_edf("S"+index_person+"R01.edf")

num person
001
num person
002
num person
003
num person
004
num person
005
num person
006
num person
007
num person
008
num person
009
num person
010
num person
011
num person
012
num person
013
num person
014
num person
015
num person
016
num person
017
num person
018
num person
019
num person
020
num person
021
num person
022
num person
023
num person
024
num person
025
num person
026
num person
027
num person
028
num person
029
num person
030
num person
031
num person
032
num person
033
num person
034
num person
035
num person
036
num person
037
num person
038
num person
039
num person
040
num person
041
num person
042
num person
043
num person
044
num person
045
num person
046
num person
047
num person
048
num person
049
num person
050
num person
051
num person
052
num person
053
num person
054
num person
055
num person
056
num person
057
num person
058
num person
059
num person
060
num person
061
num person
062
num person
063
num person
064
num person
065
num person
066
num person

IOError: [Errno 2] No such file or directory: 'S094R01.edf'

In [25]:
channel_available=[31,35,48,52,8,12,22,61] #index of Channels availables
for person in range(1,110):
    if person==89: #ruined data
        continue;
        
    if (person<=9):
        index_person="00"+str(person)
    elif (person<=99):
        index_person="0"+str(person)
    print "num person"  
    print index_person     
    ###baseline data of eye activity
    #no step, all the same
    data=load_edf("S"+index_person+"R01.edf")
    X=data[0][channel_available]
    data=load_edf("S"+index_person+"R02.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    print "baseline num"
    print X.shape
    num_wins=int((X.shape[1]-10)/5)+1
    win_channel_sample_baseline=[]
    win_channel_fft_baseline=[]
    #(batch_size,num channels,len of fft=len of sample =10)
    num_win=0
    while(num_win<num_wins):
        win_channel_sample_baseline.append(X[:,num_win*5:num_win*5+10])
        num_win+=1
    for num,channel_sample in enumerate(win_channel_sample_baseline):
        real=np.fft.fft(channel_sample,axis=1).real[:,:6]
        image=np.fft.fft(channel_sample,axis=1).imag[:,:6]
        c=np.zeros(shape=(len(channel_available),12),dtype=np.float32)
        c[:,::2]=image
        c[:,1::2]=real
        win_channel_fft_baseline.append(c)
    win_label_baseline=np.repeat(1,num_wins).tolist()#1 represent baseline
    win_channel_sample_baseline=np.array(win_channel_sample_baseline,dtype="float32")
    win_channel_fft_baseline=np.array(win_channel_fft_baseline,dtype="float32")
    print "baseline num final"
    print win_channel_sample_baseline.shape

    ### task 1:open and close left or right fist
    data=load_edf("S"+index_person+"R03.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
    data=load_edf("S"+index_person+"R07.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("S"+index_person+"R11.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    print "task1 num"
    print X.shape
    print time.shape
    win_channel_sample_task1=[]
    win_channel_fft_task1=[]
    win_label_task1=[]
    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+10-1]<=annot[0]+annot[1])&(time[i+10-1]>annot[0])):
            win_channel_sample_task1.append(X[:,i:i+10])
            win_label_task1.append(1)
            i+=5
            if(i+9>=i_max):
                break
            

    win_channel_sample_task1=np.array(win_channel_sample_task1,dtype="float32")
    for channel_sample in win_channel_sample_task1:
        real=np.fft.fft(channel_sample,axis=1).real[:,:6]
        image=np.fft.fft(channel_sample,axis=1).imag[:,:6]
        c=np.zeros(shape=(len(channel_available),12),dtype=np.float32)
        c[:,::2]=image
        c[:,1::2]=real
        win_channel_fft_task1.append(c)  
    win_channel_fft_task1=np.array(win_channel_fft_task1,dtype="float32")
    print "task1 num final"
    print win_channel_sample_task1.shape
    ###task 2 : imagery
    def label(name):
        if(name[0]=='T0'):
            return 1
        if((name[0]=='T1')|(name[0]=='T2')):
            return 2
        print name
        print "problem!"
        return 0

    data=load_edf("S"+index_person+"R04.edf")
    X=data[0][channel_available]
    time=data[3]
    annotation=data[4]
    data=load_edf("S"+index_person+"R08.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    data=load_edf("S"+index_person+"R12.edf")
    X=np.concatenate((X,data[0][channel_available]),axis=1)
    time=np.concatenate((time,data[3]),axis=0)
    annotation+=data[4]
    print "task2.shape"
    print X.shape
    print time.shape
    win_channel_sample_task2=[]
    win_channel_fft_task2=[]
    win_label_task2=[]
    i=0
    i_max=X.shape[1]
    for annot in annotation:
        if (annot[0]==0.0):
            while(time[i]!=0.0):
                i+=1
        while(time[i]<annot[0]):
            i+=1
        while((time[i+10-1]<=annot[0]+annot[1])&(time[i+10-1]>annot[0])):
            win_channel_sample_task2.append(X[:,i:i+10])
            win_label_task2.append(label(annot[2]))
            i+=5
            if(i+9>=i_max):
                break
            
    win_channel_sample_task2=np.array(win_channel_sample_task2,dtype="float32")
    for channel_sample in win_channel_sample_task2:
        real=np.fft.fft(channel_sample,axis=1).real[:,:6]
        image=np.fft.fft(channel_sample,axis=1).imag[:,:6]
        c=np.zeros(shape=(len(channel_available),12),dtype=np.float32)
        c[:,::2]=image
        c[:,1::2]=real
        win_channel_fft_task2.append(c) 
    win_channel_fft_task2=np.array(win_channel_fft_task2,dtype="float32")
    print "task2 num final"
    print win_channel_sample_task2.shape
    #input to network
    win_channel_sample=np.concatenate((win_channel_sample_baseline,win_channel_sample_task1,win_channel_sample_task2),axis=0)
    win_channel_fft=np.concatenate((win_channel_fft_baseline,win_channel_fft_task1,win_channel_fft_task2),axis=0)
    win_channel_fft=np.expand_dims(win_channel_fft,axis=3)
    win_label=np.array(win_label_baseline+win_label_task1+win_label_task2)
    
    num_window=win_channel_sample.shape[0]
    random_index=range(num_window)
    np.random.shuffle(random_index)
    win_channel_sample=win_channel_sample[random_index]
    win_channel_fft=win_channel_fft[random_index]
    win_label=win_label[random_index]
    print win_channel_fft
#     print "win_channel_sample"
#     print win_channel_sample.shape
#     print "win_cahnnel_fft"
#     print win_channel_fft.shape
#     print "win_label"
#     print win_label.shape
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"

num person
001
baseline num
(8, 19520)
baseline num final
(3903, 8, 10)
task1 num
(8, 60000)
(60000,)
task1 num final
(11835, 8, 10)
task2.shape
(8, 60000)
(60000,)
task2 num final
(11835, 8, 10)
[[[[  0.00000000e+00]
   [ -1.15000000e+02]
   [ -8.53425980e+01]
   ..., 
   [ -3.26237917e-01]
   [ -6.75015599e-14]
   [  3.00000000e+00]]

  [[  0.00000000e+00]
   [  5.10000000e+01]
   [ -8.63794098e+01]
   ..., 
   [  1.72811527e+01]
   [ -7.81597009e-14]
   [  9.00000000e+00]]

  [[  0.00000000e+00]
   [  1.29000000e+02]
   [ -2.54047653e+02]
   ..., 
   [  3.60729485e+01]
   [  7.10542736e-15]
   [  4.30000000e+01]]

  ..., 
  [[  0.00000000e+00]
   [  1.03000000e+02]
   [ -1.72799301e+02]
   ..., 
   [  3.93819656e+01]
   [ -6.39488462e-14]
   [  3.50000000e+01]]

  [[  0.00000000e+00]
   [ -7.05000000e+02]
   [  5.78756905e+01]
   ..., 
   [  2.17376208e+00]
   [ -1.01252340e-13]
   [  1.50000000e+01]]

  [[  0.00000000e+00]
   [  3.47000000e+02]
   [ -2.19658340e+02]
   ..., 
   [  

baseline num
(8, 19520)
baseline num final
(3903, 8, 10)
task1 num
(8, 60000)
(60000,)
task1 num final
(11835, 8, 10)
task2.shape
(8, 60000)
(60000,)
task2 num final
(11835, 8, 10)
[[[[  0.00000000e+00]
   [ -6.29000000e+02]
   [  1.06278351e+02]
   ..., 
   [ -3.67730217e+01]
   [  5.68434189e-14]
   [ -5.10000000e+01]]

  [[  0.00000000e+00]
   [ -5.06000000e+02]
   [  1.46341232e+02]
   ..., 
   [ -4.46656303e+01]
   [  2.84217094e-14]
   [ -4.20000000e+01]]

  [[  0.00000000e+00]
   [ -3.51000000e+02]
   [ -3.06833458e+01]
   ..., 
   [  4.76799726e+00]
   [ -7.81597009e-14]
   [  5.10000000e+01]]

  ..., 
  [[  0.00000000e+00]
   [ -4.01000000e+02]
   [ -6.68762922e-01]
   ..., 
   [  5.31056976e+00]
   [  1.06581410e-13]
   [ -4.10000000e+01]]

  [[  0.00000000e+00]
   [ -1.01800000e+03]
   [  2.66897949e+02]
   ..., 
   [ -9.78074646e+01]
   [  1.70530257e-13]
   [ -1.56000000e+02]]

  [[  0.00000000e+00]
   [ -2.26000000e+02]
   [ -6.82515564e+01]
   ..., 
   [ -2.60090904e+01]

KeyboardInterrupt: 

In [54]:
channel_available=[31,35,48,52,8,12,22,61] #index of Channels availables


index_person='034'        
print index_person     
###baseline data of eye activity
#no step, all the same
data=load_edf("S"+index_person+"R01.edf")
X=data[0][channel_available]
data=load_edf("S"+index_person+"R02.edf")
X=np.concatenate((X,data[0][channel_available]),axis=1)
print X.shape
num_wins=int((X.shape[1]-10)/5)+1
win_channel_sample_baseline=[]
win_channel_fft_baseline=[]
#(batch_size,num channels,len of fft=len of sample =10)
num_win=0
while(num_win<num_wins):
    win_channel_sample_baseline.append(X[:,num_win*5:num_win*5+10])
    num_win+=1
for num,channel_sample in enumerate(win_channel_sample_baseline):
    real=np.fft.fft(channel_sample,axis=1).real[:,:6]
    image=np.fft.fft(channel_sample,axis=1).imag[:,:6]
    c=np.zeros(shape=(len(channel_available),12),dtype=np.float32)
    c[:,::2]=image
    c[:,1::2]=real
    win_channel_fft_baseline.append(c)
win_label_baseline=np.repeat(1,num_wins).tolist()#1 represent baseline
win_channel_sample_baseline=np.array(win_channel_sample_baseline,dtype="float32")
win_channel_fft_baseline=np.array(win_channel_fft_baseline,dtype="float32")


### task 1:open and close left or right fist
data=load_edf("S"+index_person+"R03.edf")
X=data[0][channel_available]
time=data[3]
annotation=data[4]
data=load_edf("S"+index_person+"R07.edf")
X=np.concatenate((X,data[0][channel_available]),axis=1)
time=np.concatenate((time,data[3]),axis=0)
annotation+=data[4]
data=load_edf("S"+index_person+"R11.edf")
X=np.concatenate((X,data[0][channel_available]),axis=1)
time=np.concatenate((time,data[3]),axis=0)
annotation+=data[4]
print X.shape
print time.shape
win_channel_sample_task1=[]
win_channel_fft_task1=[]
win_label_task1=[]
i=0
i_max=X.shape[1]
for annot in annotation:
    if (annot[0]==0.0):
        while(time[i]!=0.0):
            i+=1
    while(time[i]<annot[0]):
        i+=1
    while((time[i+10-1]<=annot[0]+annot[1])&(time[i+10-1]>annot[0])):
        win_channel_sample_task1.append(X[:,i:i+10])
        win_label_task1.append(1)
        i+=5
        if(i+9>=i_max):
            break
            


win_channel_sample_task1=np.array(win_channel_sample_task1,dtype="float32")
for channel_sample in win_channel_sample_task1:
    real=np.fft.fft(channel_sample,axis=1).real[:,:6]
    image=np.fft.fft(channel_sample,axis=1).imag[:,:6]
    c=np.zeros(shape=(len(channel_available),12),dtype=np.float32)
    c[:,::2]=image
    c[:,1::2]=real
    win_channel_fft_task1.append(c)  
win_channel_fft_task1=np.array(win_channel_fft_task1,dtype="float32")

###task 2 : imagery
def label(name):
    if(name[0]=='T0'):
        return 1
    if((name[0]=='T1')|(name[0]=='T2')):
        return 2
    print name
    print "problem!"
    return 0

data=load_edf("S"+index_person+"R04.edf")
X=data[0][channel_available]
time=data[3]

annotation=data[4]
data=load_edf("S"+index_person+"R08.edf")
X=np.concatenate((X,data[0][channel_available]),axis=1)

time=np.concatenate((time,data[3]),axis=0)
annotation+=data[4]
data=load_edf("S"+index_person+"R12.edf")
X=np.concatenate((X,data[0][channel_available]),axis=1)
time=np.concatenate((time,data[3]),axis=0)
annotation+=data[4]
print X.shape
print time.shape
win_channel_sample_task2=[]
win_channel_fft_task2=[]
win_label_task2=[]
i=0
i_max=X.shape[1]
for annot in annotation:
    if (annot[0]==0.0):
        while(time[i]!=0.0):
            i+=1
    while(time[i]<annot[0]):
        i+=1
    while((time[i+10-1]<=annot[0]+annot[1])&(time[i+10-1]>annot[0])):
        win_channel_sample_task2.append(X[:,i:i+10])
        win_label_task2.append(label(annot[2]))
        i+=5
        if(i+9>=i_max):
            break
            
win_channel_sample_task2=np.array(win_channel_sample_task2,dtype="float32")
for channel_sample in win_channel_sample_task2:
    real=np.fft.fft(channel_sample,axis=1).real[:,:6]
    image=np.fft.fft(channel_sample,axis=1).imag[:,:6]
    c=np.zeros(shape=(len(channel_available),12),dtype=np.float32)
    c[:,::2]=image
    c[:,1::2]=real
    win_channel_fft_task2.append(c) 
win_channel_fft_task2=np.array(win_channel_fft_task2,dtype="float32")

#input to network
win_channel_sample=np.concatenate((win_channel_sample_baseline,win_channel_sample_task1,win_channel_sample_task2),axis=0)
win_channel_fft=np.concatenate((win_channel_fft_baseline,win_channel_fft_task1,win_channel_fft_task2),axis=0)
win_channel_fft=np.expand_dims(win_channel_fft,axis=3)
win_label=np.array(win_label_baseline+win_label_task1+win_label_task2)

num_window=win_channel_sample.shape[0]
random_index=range(num_window)
np.random.shuffle(random_index)
win_channel_sample=win_channel_sample[random_index]
win_channel_fft=win_channel_fft[random_index]
win_label=win_label[random_index]
print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~`"
print num_window
print win_channel_sample.shape
print win_channel_fft.shape
print win_label.shape
print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~`"
# print person
# print win_channel_sample.shape
# for i in range(539):
#     win_channel_sample_batch=win_channel_sample[i*222:(i+1)*222,:,:]
#     sample_channel_win_batch=np.swapaxes(win_channel_sample_batch,0,2)
#     sample_win_channel_batch=np.swapaxes(sample_channel_win_batch,1,2)
#     sample_win_channel_batch=sample_win_channel_batch.tolist()
#     win_channel_fft_batch=win_channel_fft[i*222:(i+1)*222,:,:,:]
#     win_label_batch=win_label[i*222:(i+1)*222]
#     print len(sample_win_channel_batch[0])
#     print i





034
(8, 19520)
(8, 59040)
(59040,)
(8, 59200)
(59200,)
27306
(27306, 8, 10)
(27306, 8, 12, 1)
(27306,)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~`


In [51]:
result=load_edf("S"+"005"+"R08.edf")
print result[0]
print result[0].shape
print "----------------"
print result[1]
print "----------------"
print result[2]
print "----------------"
print result[3]
print "----------------"
print result[4]

for i in result[3]:
    if (i==118.9+4.1):
        print "yes"

[[ 26. -41.  48. ..., -82.  24. -20.]
 [-58. -18.  -7. ...,  39.   8. -17.]
 [-81.  -2.   3. ...,  64. -15. -43.]
 ..., 
 [ -7.  17. -64. ...,   2. -43.  46.]
 [ -2.  14. -79. ...,  -4. -50.  44.]
 [-31.   1. -44. ...,  13. -33.  10.]]
(64, 19680)
----------------
160.0
----------------
['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..']
----------------
[  0.00000000e+00   6.25000000e-03   1.25000000e-02 ...,   1.22981250e+02
   1.22987500e+02   1.22993750e+02]
----------------
[(0.0, 4.1, [u'T0']), (4.1, 4.1, [u'T2']), (8.2, 4.1, [